In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dic = pickle.load(fo, encoding='bytes')
    return dic

def load_data(path=None):
    if path is None:
        print("none")
        cifar = keras.datasets.cifar10
        return cifar.load_data()
    else:
        d=[]
        l=[]
        for i in range(5):
            batch = unpickle(path + "/data_batch_" + str(i+1))
            d.append(batch[b'data'].reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1))
            l.append(np.asarray(batch[b'labels']).reshape((10000,1)))
        train_images = np.concatenate( [ d[i] for i in range(5) ])
        train_labels = np.concatenate( [ l[i] for i in range(5) ])
        batch = unpickle(path + "/test_batch")
        test_images = batch[b'data'].reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
        test_labels = np.asarray(batch[b'labels']).reshape((10000,1))
        return (train_images, train_labels), (test_images, test_labels)
                 
def normalize_data(data, mu=None, sigma=None, epsilon=10e-8):
    epsilon = 10e-8
    if mu is None:
        mu = np.mean(data, axis=0)
    if sigma is None:
        sigma = np.sqrt(np.var(data, axis=0))
    norm_data = (data-mu)/(sigma+epsilon)
    return norm_data, mu, sigma

def unnormalize_data(norm_data, mu, sigma, epsilon=10e-8):
    data = norm_data*(sigma+epsilon) + mu
    return data
        
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label.reshape((true_label.shape[0],))[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label.reshape((true_label.shape[0],))[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1]) 
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
(train_images, train_labels), (test_images, test_labels) = load_data()
#(train_images, train_labels), (test_images, test_labels) = load_data("data/cifar-10-batches-py")

In [ ]:
class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 
               'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

In [ ]:
train_images, mu_X, sigma_X = normalize_data(train_images)
test_images, mu_X, sigma_X = normalize_data(test_images, mu_X, sigma_X)

In [ ]:
train_images.shape

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(32,32,3)),
    keras.layers.Dense(1024),
    keras.layers.LeakyReLU(alpha=0.01),
    keras.layers.Dense(128),
    keras.layers.LeakyReLU(alpha=0.01),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
sgd = optimizers.SGD(lr=0.01, decay=0, momentum=0.9, nesterov=False)
keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=10e-7, decay=0.0)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, batch_size=1024, epochs=5)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)


In [ ]:
predictions = model.predict(test_images)

In [ ]:
i = 100
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, unnormalize_data(test_images, mu_X, sigma_X)/255.)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)